#**SQLite3**

In [ ]:
import sqlite3

In [ ]:
conn = sqlite3.connect("school.db")
cursor = conn.cursor()

In [ ]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS students (
    student_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    age INTEGER,
    grade TEXT
)
""")

In [ ]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS subjects (
    subject_id INTEGER PRIMARY KEY AUTOINCREMENT,
    subject_name TEXT NOT NULL,
    student_id INTEGER,
    score INTEGER,
    FOREIGN KEY(student_id) REFERENCES students(student_id)
)
""")


In [ ]:
cursor.executemany("""
INSERT INTO students (name, age, grade) VALUES (?, ?, ?)
""", [
    ("Ali", 17, "11th"),
    ("Sara", 16, "10th"),
    ("Youssef", 17, "11th"),
    ("Mona", 18, "12th")
])

In [ ]:
cursor.executemany("""
INSERT INTO subjects (subject_name, student_id, score) VALUES (?, ?, ?)
""", [
    ("Math", 1, 85),
    ("Science", 1, 90),
    ("Math", 2, 78),
    ("Science", 2, 88),
    ("Math", 3, 92),
    ("Science", 3, 80),
    ("Math", 4, 60),
    ("Science", 4, 72)
])


In [ ]:
conn.commit()

**Subjects per student**

In [ ]:
cursor.execute("""
SELECT s.name, COUNT(sub.subject_id) as subject_count
FROM students s
LEFT JOIN subjects sub ON s.student_id = sub.student_id
GROUP BY s.name
""")

result = cursor.fetchall()
print(result)
for i in result:
    print(i)

[('Ali', 2), ('Mona', 2), ('Sara', 2), ('Youssef', 2)]
('Ali', 2)
('Mona', 2)
('Sara', 2)
('Youssef', 2)


**Math score > average**

In [ ]:
cursor.execute("""
SELECT s.name, sub.score
FROM students s
JOIN subjects sub ON s.student_id = sub.student_id
WHERE sub.subject_name = 'Math' AND sub.score > (
    SELECT AVG(score) FROM subjects WHERE subject_name = 'Math'
)
""")
print(cursor.fetchall())

[('Ali', 85), ('Youssef', 92)]


**Names starting with 'A'**

In [ ]:
cursor.execute("""
SELECT name, grade FROM students WHERE name LIKE 'A%'
""")
print(cursor.fetchall())

[('Ali', '11th')]


**Highest scorer in each subject**

In [ ]:
cursor.execute("""
SELECT sub.subject_name, s.name, sub.score
FROM subjects sub

JOIN students s ON s.student_id = sub.student_id

WHERE (sub.subject_name, sub.score) IN (
    SELECT subject_name, MAX(score)
    FROM subjects
    GROUP BY subject_name
)
""")
print(cursor.fetchall())

[('Science', 'Ali', 90), ('Math', 'Youssef', 92)]


# **SQL Injection**

In [ ]:
import sqlite3

conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

In [ ]:
cursor.execute("""
CREATE TABLE users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT,
    password TEXT
)
""")

In [ ]:
cursor.execute("INSERT INTO users (username, password) VALUES ('admin', 'admin123')")
cursor.execute("INSERT INTO users (username, password) VALUES ('user', '1234')")
conn.commit()

In [ ]:
username = input("Username: ")
password = input("Password: ")

query = f"SELECT * FROM users WHERE username = '{username}' AND password = '{password}'"
print(f"[DEBUG] Executing: {query}")
cursor.execute(query)

user = cursor.fetchone()
if user:
    print("✅ Login successful!")
else:
    print("❌ Invalid credentials!")

Username: admin
Password: ' or 1=1 --
[DEBUG] Executing: SELECT * FROM users WHERE username = 'admin' AND password = '' or 1=1 --'
✅ Login successful!


In [ ]:
username = input("Username: ")
password = input("Password: ")

query = "SELECT * FROM users WHERE username = ? AND password = ?"
cursor.execute(query, (username, password))

user = cursor.fetchone()
if user:
    print("✅ Login successful!")
else:
    print("❌ Invalid credentials!")

Username: admin
Password: ' or 1=1 --
❌ Invalid credentials!


# **Chinook**

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("Chinook_Sqlite.sqlite")

In [ ]:
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)

In [ ]:
tables

,name
0,Album
1,Artist
2,Customer
3,Employee
4,Genre
5,Invoice
6,InvoiceLine
7,MediaType
8,Playlist
9,PlaylistTrack


In [ ]:
df_customers = pd.read_sql("SELECT * FROM Customer", conn)

In [ ]:
df_customers.head()

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [ ]:
print("Shape:", df_customers.shape)

Shape: (59, 13)


In [ ]:
# نوع كل عمود
print("\nInfo:")
df_customers.info()


Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CustomerId    59 non-null     int64 
 1   FirstName     59 non-null     object
 2   LastName      59 non-null     object
 3   Company       10 non-null     object
 4   Address       59 non-null     object
 5   City          59 non-null     object
 6   State         30 non-null     object
 7   Country       59 non-null     object
 8   PostalCode    55 non-null     object
 9   Phone         58 non-null     object
 10  Fax           12 non-null     object
 11  Email         59 non-null     object
 12  SupportRepId  59 non-null     int64 
dtypes: int64(2), object(11)
memory usage: 6.1+ KB


In [ ]:
df_customers.isnull().sum()

,0
CustomerId,0
FirstName,0
LastName,0
Company,49
Address,0
City,0
State,29
Country,0
PostalCode,4
Phone,1


In [ ]:
df_customers.drop(columns=['Fax'], inplace=True)

In [ ]:
df_customers.fillna({'Company': 'Individual'}, inplace=True)

In [ ]:
df_customers['Email'] = df_customers['Email'].str.lower()

In [ ]:
df_customers.rename(columns={'SupportRepId': 'SupportAgentId'}, inplace=True)

In [ ]:
df_customers['Country'].value_counts().head(10)

,count
Country,
USA,13
Canada,8
France,5
Brazil,5
Germany,4
United Kingdom,3
Portugal,2
Czech Republic,2
India,2


In [ ]:
df_customers['SupportAgentId'].value_counts()

,count
SupportAgentId,
3,21
4,20
5,18


In [ ]:
df_customers[df_customers['Company'] == "Individual"].shape[0]

49